In [67]:
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

## Install libraries

In [68]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [69]:
video_id = "eur8dUO9mvE" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

If you're building AI agents, you've probably heard about MCP or Model Context Protocol. MCP is a new open source standard to connect your agents to data sources such as databases or APIs. MCP consists of multiple components. The most important ones are the host, the client, and the server. So let's break it down. At the very top you would have your MCP host. Your MCP host will include an MCP client. And it could also include multiple clients. The MCP host could be an application such as a chat app. It could also be a code assistant in your IDE, and much more. The MCP host will connect to an MCP server. It can actually connect to multiple MCP servers as well. It doesn't matter how many MCP servers you connect to your MCP host or client. The MCP host and servers will connect over each other through the MCP protocol. The MCP protocol is a transport layer in the middle. Whenever your MCP host or client needs a tool, it's going to connect to the MCP server. The MCP server will then connect

In [70]:
transcript_list

[{'text': "If you're building AI agents, you've probably heard about MCP or Model Context Protocol.",
  'start': 0.4,
  'duration': 4.299},
 {'text': 'MCP is a new open source standard to connect your agents to data sources such as databases or APIs.',
  'start': 5.24,
  'duration': 5.699},
 {'text': 'MCP consists of multiple components.',
  'start': 11.68,
  'duration': 1.919},
 {'text': 'The most important ones are the host, the client, and the server.',
  'start': 13.94,
  'duration': 3.08},
 {'text': "So let's break it down.", 'start': 17.22, 'duration': 1.0},
 {'text': 'At the very top you would have your MCP host.',
  'start': 19.53,
  'duration': 2.459},
 {'text': 'Your MCP host will include an MCP client.',
  'start': 22.57,
  'duration': 2.8},
 {'text': 'And it could also include multiple clients.',
  'start': 25.91,
  'duration': 1.9},
 {'text': 'The MCP host could be an application such as a chat app.',
  'start': 28.37,
  'duration': 4.039},
 {'text': 'It could also be a co

## Step 1b - Indexing (Text Splitting)

In [71]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [72]:
len(chunks)

4

In [73]:
# chunks[100]

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [74]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(chunks, embeddings)

In [75]:
vector_store.index_to_docstore_id

{0: 'ad45e536-2023-4db2-8886-a5cbcebc2aea',
 1: '785175a3-6ca4-4759-ad13-3576c75d58e3',
 2: 'ab4c2238-24c7-4d6a-8aa3-84e27123b40a',
 3: 'd163f1a9-3291-409f-8e79-d5f3f844068e'}

In [76]:
vector_store.get_by_ids(['6879e71a-fad7-410a-a3fc-10293ff7998c'])

[]

## Step 2 - Retrieval

In [77]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [78]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000208C18BBB10>, search_kwargs={'k': 4})

In [79]:
retriever.invoke('What is MCP?')

[Document(id='ad45e536-2023-4db2-8886-a5cbcebc2aea', metadata={}, page_content="If you're building AI agents, you've probably heard about MCP or Model Context Protocol. MCP is a new open source standard to connect your agents to data sources such as databases or APIs. MCP consists of multiple components. The most important ones are the host, the client, and the server. So let's break it down. At the very top you would have your MCP host. Your MCP host will include an MCP client. And it could also include multiple clients. The MCP host could be an application such as a chat app. It could also be a code assistant in your IDE, and much more. The MCP host will connect to an MCP server. It can actually connect to multiple MCP servers as well. It doesn't matter how many MCP servers you connect to your MCP host or client. The MCP host and servers will connect over each other through the MCP protocol. The MCP protocol is a transport layer in the middle. Whenever your MCP host or client needs a

## Step 3 - Augmentation

In [80]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

In [81]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [82]:
question          = "What is there in this particular video?"
retrieved_docs    = retriever.invoke(question)

In [83]:
retrieved_docs

[Document(id='ab4c2238-24c7-4d6a-8aa3-84e27123b40a', metadata={}, page_content="multiple MCP servers or just a single one. Let's assume our MCP client and host is a chat app, and you ask a question such as, what is the weather like in a certain location or how many customers do I have? The MCP host will need to retrieve tools from the MCP server. The MCP server will then conclude and tell which tools are available. From the MCP host, you would then have to connect to the large language model and send over your question plus the available tools. If all is well, the LLM will reply and tell you which tools to use. Once the MCP host and client knows which tools to use, it knows which MCP servers to call. So when it calls the MCP server in order to get a tool result, the MCP server will be responsible for executing something that goes to a database, to an API, or a local piece of code, and of course, there could be subsequent calls to MCP servers. The MCP server will apply with a response, 

In [84]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"multiple MCP servers or just a single one. Let's assume our MCP client and host is a chat app, and you ask a question such as, what is the weather like in a certain location or how many customers do I have? The MCP host will need to retrieve tools from the MCP server. The MCP server will then conclude and tell which tools are available. From the MCP host, you would then have to connect to the large language model and send over your question plus the available tools. If all is well, the LLM will reply and tell you which tools to use. Once the MCP host and client knows which tools to use, it knows which MCP servers to call. So when it calls the MCP server in order to get a tool result, the MCP server will be responsible for executing something that goes to a database, to an API, or a local piece of code, and of course, there could be subsequent calls to MCP servers. The MCP server will apply with a response, which you can send back to the LLM. And finally, you should be able to get your

In [85]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [86]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      multiple MCP servers or just a single one. Let's assume our MCP client and host is a chat app, and you ask a question such as, what is the weather like in a certain location or how many customers do I have? The MCP host will need to retrieve tools from the MCP server. The MCP server will then conclude and tell which tools are available. From the MCP host, you would then have to connect to the large language model and send over your question plus the available tools. If all is well, the LLM will reply and tell you which tools to use. Once the MCP host and client knows which tools to use, it knows which MCP servers to call. So when it calls the MCP server in order to get a tool result, the MCP server will be responsible for executing something that goes to a database, to an API, or a local piece of c

## Step 4 - Generation

In [87]:
answer = llm.invoke(final_prompt)
print(answer.content)

This video describes the Model Context Protocol (MCP), a new open-source standard for connecting AI agents to data sources like databases or APIs.  It explains the three main components: the host, client, and server, and how they interact using the MCP protocol as a transport layer.  The video uses the example of a chat app to illustrate how MCP works in practice, showing how a question is routed through the system to access tools and retrieve information from various sources (databases, APIs, local code).  The video also advises viewers building AI agents to consider using the MCP protocol.


## Building a Chain

In [88]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [89]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [90]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [91]:
parallel_chain.invoke('who is Demis')

{'context': "through the MCP protocol. The MCP protocol is a transport layer in the middle. Whenever your MCP host or client needs a tool, it's going to connect to the MCP server. The MCP server will then connect to, for example, a database. And it doesn't matter if this is a relational database or a NoSQL database. It could also connect to APIs. And also the API standard doesn't really matter. Finally, it could also connect to data sources such as a local file type or maybe code. This is especially useful when you're building something like a code assistant in your IDE. Let's look at an example of how to use MCP in practice. We still have the three components. We would have our MCP host and client, of course, we also have a large language model, and finally, we have our MCP servers, and these could be multiple MCP servers or just a single one. Let's assume our MCP client and host is a chat app, and you ask a question such as, what is the weather like in a certain location or how many 

In [92]:
parser = StrOutputParser()

In [93]:
main_chain = parallel_chain | prompt | llm | parser

In [94]:
main_chain.invoke('What is explained in this video?')

"The video explains the Model Context Protocol (MCP), a new open-source standard for connecting AI agents to data sources like databases or APIs.  It details the MCP's components (host, client, and server), how they interact, and provides an example using a chat app to illustrate how MCP works with LLMs to retrieve information from various sources."